In [ ]:
import pandas as pd
import numpy as np
import itertools
import re
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
train = pd.read_json('../input/train.json')

In [ ]:
train.head()

In [ ]:
test = pd.read_json('../input/test.json')

In [ ]:
cuisine = train['cuisine'].unique()
samples = train['cuisine'].value_counts()

In [ ]:
ingredients = train['ingredients']

In [ ]:
# Separate each type of cuisine and calculate the frequency each ingredient appears under each cuisine type
# Also Convert plural forms into singular as much as can

In [ ]:
def list_str_plural(lst):
    str_cov = " ".join(lst)
    middle1 = re.sub(r'\d+', '', str_cov)     # Remove any digit from ingredients
    middle2 = middle1.replace('%','')
    middle3 = middle1.split(' ')
    middle4 = list(map(lambda word: WordNetLemmatizer().lemmatize(word.lower()),middle3))
    return ' '.join(middle4)

ingredients_new = ingredients.apply(list_str_plural)
train['ingredients'] = ingredients_new

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(cuisine)
le.transform(train['cuisine'])

In [ ]:
tfidf = TfidfVectorizer(binary=True,use_idf=False)
X = tfidf.fit_transform(train['ingredients'])


In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import GridSearchCV
param = {'C':[0.1,1,5, 10, 100],'gamma':[1,0.1,0.01,0.001,0.0001]}

In [ ]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
lsvc.fit(X,train['cuisine'])

In [ ]:
# Test on test set

In [ ]:
test['ingredients'] = test['ingredients'].apply(list_str_plural)

In [ ]:
Xtest = tfidf.transform(test['ingredients'])

In [ ]:
prediction = lsvc.predict(Xtest)
test['cuisine'] = prediction

In [ ]:
new_test = pd.DataFrame(test[['id','cuisine']], columns=['id','cuisine'])
new_test.set_index('id')
prediction = new_test.to_csv('prediction.csv',index=False)